## **Librerías**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.options.display.max_columns = False

## **1. Datos**

In [3]:
df = pd.read_csv('./Datos/titanic.csv')

In [4]:
df.sample()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
365,1,3,Mrs. (Mantoura Boulos) Moussa,female,35.0,0,0,7.2292


## **2. EDA**

### **Exploración inicial**

In [20]:
df.head()

,ID,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,0,3,male,22.0,1,0,7.2500
1,1,1,1,female,38.0,1,0,71.2833
2,2,1,3,female,26.0,0,0,7.9250
3,3,1,1,female,35.0,1,0,53.1000
4,4,0,3,male,35.0,0,0,8.0500


In [9]:
df.reset_index(inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       887 non-null    int64  
 1   Survived                 887 non-null    int64  
 2   Pclass                   887 non-null    int64  
 3   Sex                      887 non-null    object 
 4   Age                      887 non-null    float64
 5   Siblings/Spouses Aboard  887 non-null    int64  
 6   Parents/Children Aboard  887 non-null    int64  
 7   Fare                     887 non-null    float64
dtypes: float64(2), int64(5), object(1)
memory usage: 55.6+ KB


In [22]:
df.describe()

,ID,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
count,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.00000
mean,443.000000,0.385569,2.305524,29.471443,0.525366,0.383315,32.30542
std,256.199141,0.487004,0.836662,14.121908,1.104669,0.807466,49.78204
min,0.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.00000
25%,221.500000,0.000000,2.000000,20.250000,0.000000,0.000000,7.92500
50%,443.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.45420
75%,664.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.13750
max,886.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.32920


### **Limpieza de los datos**

In [18]:
df.drop('Name', axis=1, inplace=True)

In [19]:
df

,ID,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,0,3,male,22.0,1,0,7.2500
1,1,1,1,female,38.0,1,0,71.2833
2,2,1,3,female,26.0,0,0,7.9250
3,3,1,1,female,35.0,1,0,53.1000
4,4,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,882,0,2,male,27.0,0,0,13.0000
883,883,1,1,female,19.0,0,0,30.0000
884,884,0,3,female,7.0,1,2,23.4500
885,885,1,1,male,26.0,0,0,30.0000


In [15]:
df.rename(columns = {
    'index': 'ID',
    'Siblings/Spouses Aboard': 'SibSp',
    'Parents/Children Aboard': 'Parch'
}, inplace=True)